In [4]:
# Basic Modules
import numpy as np
import pandas as pd
from scipy.cluster.vq import *
import operator
import matplotlib
reload(matplotlib)
matplotlib.use('Agg')
from matplotlib import pyplot as plt   
import pickle as pkl
import shelve
import re
from collections import Counter, defaultdict, OrderedDict
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import metrics#v_measure_score
import scipy
from sklearn.feature_extraction import DictVectorizer
from matplotlib.backends.backend_pdf import PdfPages
import csv
import sys
import math
from copy import deepcopy
import random
from datetime import datetime, timedelta
import pickle
import sys
from multiprocessing import Pool
import pp
import shelve
from collections import OrderedDict

# ML modules
from sklearn.ensemble import AdaBoostClassifier
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.svm import OneClassSVM
from sklearn.mixture import GMM
from sklearn.mixture import DPGMM
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB, BernoulliNB, GaussianNB
from sklearn.neural_network import BernoulliRBM as RBM

# Custom modules
from bd_wrapper import BDWrapper

In [5]:
sensor_dict = shelve.open('metadata/bacnet_devices.shelve','r')

In [6]:
buildingName = 'bonner'
naeDict = dict()
naeDict['bonner'] = ["607", "608", "609", "557", "610"]
naeDict['ap_m'] = ['514', '513','604']
naeDict['bsb'] = ['519', '568', '567', '566', '564', '565']
naeDict['ebu3b'] = ["505", "506"]
naeList = naeDict[buildingName]
missingSrcidList = list()
with open('missingsrcidlist.csv', 'rb') as fp:
    reader = csv.reader(fp)
    for row in reader:
        missingSrcidList.append(row[0])
outputFilename = 'data/fe_' + buildingName + '.pkl'

In [4]:
def extract_features(srcidList, dummy, buildingName):
    import feature_extractor as fe
    import shelve
    resultList = list()
    invalidSrcidList = list()
    for srcid in srcidList:
        try:
            filename = 'rawdata/'+buildingName+'/'+srcid+'.shelve'
        #with open('rawdata/ebu3b/'+srcid+'.pkl', 'rb') as fp:
            #ts = pickle.load(fp)
            ts = shelve.open(filename)['data']
        #print "hello"
        
            resultList.append((srcid, fe.get_features(ts)))
        #resultList.append((srcid, 10))
            ts = None
        except:
            invalidSrcidList.append(srcid)
            continue
    print invalidSrcidList
    return resultList

In [5]:
def temp_func(srcid):
    return (srcid, 10)

In [6]:
uuidList = list()
srcidList = list()
for nae in naeList:
    deviceList = sensor_dict[nae]
    for sensor in deviceList['objs'][1:]:
        h_obj = sensor['props']
        srcid = nae + '_' + str(h_obj['type']) + '_' + str(h_obj['instance'])
        if not srcid in missingSrcidList:
            srcidList.append(srcid)
print len(srcidList)

13800


In [7]:
#ncpus = 4
#rangeList = list()

#sensorsNum = len(srcidList)
#for i in range(0,ncpus):
#    rangeList.append(range(sensorsNum/ncpus*(i+1) - sensorsNum/ncpus, sensorsNum/ncpus*(i+1)))

#srcidListList = list()
#for oneRange in rangeList:
#oneList =  [srcidList[i] for i in oneRange]
#    srcidListList.append(oneList)
    
#p = Pool(ncpus)
#tempDict = dict((p.map(extract_features, srcidListList)))

In [8]:
#p = Pool(4)
#tempDict = dict((p.map(extract_features, srcidList)))

ppservers = ()
ncpus = 4
rangeList = list()

#srcidList = ['505_0_3000043', '506_0_3000026', '505_0_3000003', '506_0_3000023',  '506_0_3000027']

sensorsNum = len(srcidList)
for i in range(0,ncpus):
    rangeList.append(range(sensorsNum/ncpus*(i+1) - sensorsNum/ncpus, sensorsNum/ncpus*(i+1)))
print "=============="

jobServer = pp.Server(ncpus, ppservers=ppservers)
jobList = list()
for oneRange in rangeList:
    #print [srcidList[i] for i in oneRange]
    jobList.append(jobServer.submit(extract_features, ([srcidList[i] for i in oneRange], True, buildingName)))

resultList = list()
resultList = [0,0,0,0]
for i, job in enumerate(jobList):
    resultList[i] = job()
#r1 = jobList[0]()
#r2 = jobList[1]()
jobServer.wait()
print "=-============"
#print r1
#print r2

dictList = list()
#print resultList
for result in resultList:
#    print "result: ", result
    dictList = dictList + result
resultDict = dict(dictList)
#print resultDict

#job_server = pp.Server(ncpus, ppservers=ppservers)
#j1 = job_server.submit(extract_features, srcidList[0:1500])

['607_16_3001438', '607_16_3001609', '607_16_3002412', '607_16_3002422', '607_16_3002426', '607_16_3002429', '607_17_3002195', '607_17_3002196', '607_20_3000006', '607_20_3000007', '607_20_3000806', '607_20_3000807', '607_20_3000808', '607_20_3000810', '607_20_3000811', '607_20_3000812', '607_20_3000813', '607_20_3000814', '607_20_3000815', '607_20_3000817', '607_20_3000818', '607_20_3000819', '607_20_3000821', '607_20_3000822', '607_20_3000823', '607_20_3000824', '607_20_3000825', '607_20_3000826', '607_20_3000828', '607_20_3000829', '607_20_3000830', '607_20_3000832', '607_20_3000833', '607_20_3000834', '607_20_3000835', '607_20_3000836', '607_20_3000837', '607_20_3000839', '607_20_3000840', '607_20_3000841', '607_20_3000843', '607_20_3000844', '607_20_3000845', '607_20_3000846', '607_20_3000847', '607_20_3000848', '607_20_3000850', '607_20_3000851', '607_20_3000852', '607_20_3000854', '607_20_3000855', '607_20_3000856', '607_20_3000857', '607_20_3000858', '607_20_3000861', '607_20_3

In [9]:
with open(outputFilename, 'wb') as fp:
    pickle.dump(resultDict, fp)
print len(resultDict)

3212


In [10]:
## TEST FE
import feature_extractor 
reload(feature_extractor)
import feature_extractor as fe
import FATS


srcidDict = OrderedDict()
srcidDict['ZT'] = '506_0_3000485' # 2150 ZT
srcidDict['OCM'] = '506_1_3000497' # 2150 occ clg max
srcidDict['CS'] = '506_1_3000493' # 2150 TS
srcidDict['DC'] = '506_1_3000494' # 2150 

fat = FATS.FeatureSpace(Data=['magnitude', 'time'], featureList=[#'Mean', 'Amplitude', 'Skew', 
                                                                 #'Meanvariance', 
                                                                 #'PeriodLS' 
                                                                 #'Std', 
                                                                 #'MaxSlope', 
                                                                 'SmallKurtosis'
                                                                ])
#for pointType, srcid in srcidDict.iteritems():
#    filename = 'rawdata/ebu3b/'+srcid+'.shelve'
#    ts = shelve.open(filename)['data']
#    #print pointType, fe.get_fft(ts)
#    ts = np.asarray([ts.values, ts.index.values])
#    result = fat.calculateFeature(ts).result()
#    print pointType, result

result = extract_features(srcidDict.values(), True, buildingName)
print result


['506_0_3000485', '506_1_3000497', '506_1_3000493', '506_1_3000494']
[]


In [11]:
filename = 'rawdata/ebu3b/506_0_3000485.shelve'
ts = shelve.open(filename)['data'].values
print(len(ts))

5041


In [12]:
### RBM
print len(srcidList)
rawdataDict = dict()
#srcidList = list()
i = 0

for srcid in srcidList:
    if i%500==0:
        print i
    i += 1
    filename = 'rawdata/ebu3b/'+srcid+'.shelve'
        #with open('rawdata/ebu3b/'+srcid+'.pkl', 'rb') as fp:
            #ts = pickle.load(fp)
    rawdataDict[srcid] = shelve.open(filename)['data'].values
    #srcidList.append(srcid)
    
rbm = RBM(n_components=32)
rbmDataArr = rbm.fit_transform(rawdataDict.values())



13800
0


KeyError: 'data'